# RAP analysis environmental variable extraction automated

#### Need to change location of RAP analysis file output for external use
#### WARNING: script takes lengthly period to read all files and compute/read variables (~30 hours for 1000 mesoscale discussion)

# Unit test to determine if input CSV has all necessary information

In [ ]:
def csv_test(df):
    '''Test to make sure each MD contain a product id and a label for RAP analysis files and output to output csv'''
    if (input_csv["SPC PRODUCT ID"].isnull().sum().all() == 0) & (input_csv["WATCH PARAMETER"].isnull().sum().all() == 0):
        print('✅ CSV file contains all necessary data!')
    else:
        sys.exit('❌ Error. One or more values are missing. Please check csv file to edit/remove any dates with incomplete values.')

# Automated unit test to determine if RAP analysis files contains neccessary environmental variables

In [ ]:
def rap_variable_test(ds_raw_file, ds_mean_sea, ds_surface, ds_2m, ds_10m, ds_reflect, year, month, day, hour):
    '''Test to make due each RAP analysis file has all desired variables'''
    if ds_raw_file.get('PWAT_P0_L200_GLC0')[130, 200] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Precipitable Water Not Avalible in Dataset"
    if ds_mean_sea.mslma[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Mean Sea Level Pressure Not Avalible in Dataset"
    if ds_surface.sp[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Surface Pressure Not Avalible in Dataset"
    if ds_surface.cape[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Surface Based CAPE Not Avalible in Dataset"
    if ds_surface.cin[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Surface Based CIN Not Avalible in Dataset"
    if ds_2m.t2m[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 2-m Temperature Not Avalible in Dataset"
    if ds_2m.r2[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 2-m Relative Humidity Not Avalible in Dataset"
    if ds_2m.sh2[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 2-m Specific Humidity Not Avalible in Dataset"
    if ds_10m.u10[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 10-m U-Component Wind Not Avalible in Dataset"
    if ds_10m.v10[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 10-m V-Component Wind Not Avalible in Dataset"
    if ds_raw_file.get('VUCSH_P0_2L103_GLC0')[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 0-6 km U-Component Wind Shear Not Avalible in Dataset"
    if ds_raw_file.get('VVCSH_P0_2L103_GLC0')[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 0-6 km V-Component Wind Shear Not Avalible in Dataset"
    if ds_raw_file.get('HLCY_P0_2L103_GLC0')[0][130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 0-1 km Storm Relative Helicity Not Avalible in Dataset"
    if ds_raw_file.get('HLCY_P0_2L103_GLC0')[1][130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ 0-3 km Storm Relative Helicity Not Avalible in Dataset"
    if ds_reflect.refd[130, 230] == np.nan:
        f"{year}/{month}/{day}_{hour}00 ⚠️ Composite Reflectivity Not Avalible in Dataset"

# Download RAP analysis files and extract environmental parameters to CSV file

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from csv import writer
import urllib.request
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity, lcl, wind_speed
import sys

# Import warnings package to prevent non-critical warnings from displaying
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Read in csv file with mesoscale discussion data
input_csv = pd.read_csv("central_ok_mds_FINAL.csv")
csv_test(input_csv)

prod_id_list = list(input_csv["SPC PRODUCT ID"])
csv_watch_md_list = list(input_csv["WATCH PARAMETER"])

# Set constants
placeholder = -9999
input_csv_row = 0
 
for j in prod_id_list:
        
    # Extract whether watch (1) or md (0) was issued
    label = int(csv_watch_md_list[input_csv_row])
        
    # Create year, month, day, and hour fields based upon the Product ID 
    year = str(prod_id_list[input_csv_row][0:4])
    month = str(prod_id_list[input_csv_row][4:6])
    day = str(prod_id_list[input_csv_row][6:8])
    hour = str(prod_id_list[input_csv_row][8:10])
        
    try:
        # NCEI 13km RAP Non-Operational Archive (Long-Term)
        urllib.request.urlretrieve(f"https://www.ncei.noaa.gov/thredds/fileServer/model-rap130anl-old/{year}{month}/{year}{month}{day}/rap_130_{year}{month}{day}_{hour}00_000.grb2", 
                                       f"/home/scratch/nsonntag/eae598/rap_data/rap_130_{year}{month}{day}_{hour}00_000.grb2")
        ds_avail_check = 1
    except:
        try:
            # NCEI 13km RAP Operational Archive (Short-Term)
            urllib.request.urlretrieve(f"https://www.ncei.noaa.gov/thredds/fileServer/model-rap130anl/{year}{month}/{year}{month}{day}/rap_130_{year}{month}{day}_{hour}00_000.grb2", 
                                           f"/home/scratch/nsonntag/eae598/rap_data/rap_130_{year}{month}{day}_{hour}00_000.grb2")
            ds_avail_check = 1
        except:
            # Print Date and Message that Date Isn't Avalible in Archive
            ds_avail_check = 0
            print(f"{year}/{month}/{day}_{hour}00 ❌ RAP Analysis File Not Avalible in Dataset")
                
    # Check if dataset is available        
    if ds_avail_check == 1:
            
        # Input location of RAP analysis file
        ds = f"/home/scratch/nsonntag/eae598/rap_data/rap_130_{year}{month}{day}_{hour}00_000.grb2"
            
        # User defined array of grid points over which variable will be averaged
        # can be calc
        grid_y = [126, 126, 126, 127, 127, 127, 127, 127, 127, 127, 128, 128, 128, 128, 128, 128, 128, 129, 129, 
                  129, 129, 129, 129, 129, 129, 129, 130, 130, 130, 130, 130, 130, 130, 130, 130, 131, 131, 131, 
                  131, 131, 131, 131, 131, 131, 132, 132, 132, 132, 132, 132, 132, 132, 132, 133, 133, 133, 133, 
                  133, 133, 133, 134, 134, 134, 134, 134]
        grid_x = [228, 229, 230, 226, 227, 228, 229, 230, 231, 232, 226, 227, 228, 229, 230, 231, 232, 225, 226, 
                  227, 228, 229, 230, 231, 232, 233, 225, 226, 227, 228, 229, 230, 231, 232, 233, 225, 226, 227, 
                  228, 229, 230, 231, 232, 233, 225, 226, 227, 228, 229, 230, 231, 232, 233, 226, 227, 228, 229, 
                  230, 231, 232, 227, 228, 229, 230, 231]
            
        grid_points = np.vstack((grid_y, grid_x)).T
            
        # Open RAP analysis file for each defined level
        ds_dbz = xr.open_dataset(ds, filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 1000})
        ds_raw = xr.open_dataset(ds, engine="pynio")
        ds_ms = xr.open_dataset(ds, filter_by_keys={'typeOfLevel': 'meanSea'})
        ds_sfc = xr.open_dataset(ds, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
        ds_2 = xr.open_dataset(ds, filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2})
        ds_10 = xr.open_dataset(ds, filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 10})
        
        # Unit test for environmental variables in RAP analysis data 
        rap_variable_test(ds_raw, ds_ms, ds_sfc, ds_2, ds_10, ds_dbz, year, month, day, hour)
        
        # Set sums over user defined grid equal to zero for computation proposes
        pwat_sum = mslp_sum = cape_sum = cin_sum = t2m_sum = rh2m_sum = q2m_sum = u10_sum = v10_sum = 0
        uv10_sum = lcl_sum = shear_0_6_sum = srh_0_1_sum = srh_0_3_sum = 0
        
        # set number of points not filtered by reflectivity equal to zero
        number_points = 0
            
        # Run analysis over all user defined grid
        for i in range(len(grid_points)):
            
            # filter out point that have a radar reflectivity greater than 30 dBz
            reflectivity = ds_dbz.refd[grid_points[i,0], grid_points[i,1]]
            
            if reflectivity <= 30:
                
                # Precipitable water
                pwat_point = ds_raw.get('PWAT_P0_L200_GLC0')[grid_points[i,0], grid_points[i,1]]
                pwat_sum = pwat_sum + pwat_point
                
                # Mean sea level pressure
                mslp_point = ds_ms.mslma[grid_points[i,0], grid_points[i,1]]
                mslp_sum = mslp_sum + mslp_point
                
                # Surface pressure for bulk shear calculations
                slp_point = ds_sfc.sp[grid_points[i,0], grid_points[i,1]]
                
                # Convective available potential energy
                cape_point = ds_sfc.cape[grid_points[i,0], grid_points[i,1]]
                cape_sum = cape_sum + cape_point
                
                # Convective inhibition
                cin_point = ds_sfc.cin[grid_points[i,0], grid_points[i,1]]
                cin_sum = cin_sum + cin_point
                
                # 2-m temperature
                t2m_point = ds_2.t2m[grid_points[i,0], grid_points[i,1]]
                t2m_sum = t2m_sum + t2m_point
                
                # 2-m relative humidity
                rh2m_point = ds_2.r2[grid_points[i,0], grid_points[i,1]]
                rh2m_sum = rh2m_sum + rh2m_point
                
                # 2-m specific humidity
                q2m_point = ds_2.sh2[grid_points[i,0], grid_points[i,1]]
                q2m_sum = q2m_sum + q2m_point
                
                # 10-m u-component wind
                u10_point = ds_10.u10[grid_points[i,0], grid_points[i,1]]
                u10_sum = u10_sum + u10_point
                
                # 10-m v-component wind
                v10_point = ds_10.v10[grid_points[i,0], grid_points[i,1]]
                v10_sum = v10_sum + v10_point
                
                # Computation for 10-m wind speed
                uv10_point = wind_speed(u10_point, v10_point)
                uv10_sum = uv10_sum + uv10_point
                
                # Computation for lifted condensation level pressure
                dew_point = dewpoint_from_relative_humidity(((t2m_point - 273.15) * units('degC')), rh2m_point)
                lcl_point = lcl(((slp_point / 100)  * units('hPa')), t2m_point, dew_point)
                lcl_sum = lcl_sum + lcl_point[0]
                
                # Computation for 0-6 km shear
                u_shear_0_6_point = (ds_raw.get('VUCSH_P0_2L103_GLC0')[grid_points[i,0], grid_points[i,1]]) * units('m/s')
                v_shear_0_6_point = (ds_raw.get('VVCSH_P0_2L103_GLC0')[grid_points[i,0], grid_points[i,1]]) * units('m/s')
                shear_0_6_point = wind_speed(u_shear_0_6_point, v_shear_0_6_point)
                shear_0_6_sum = shear_0_6_sum + shear_0_6_point
                
                # 0-1 km storm relative helicity
                srh_0_1_point = ds_raw.get('HLCY_P0_2L103_GLC0')[0][grid_points[i,0], grid_points[i,1]]
                srh_0_1_sum = srh_0_1_sum + srh_0_1_point
                
                # 0-3 km storm relative helicity
                srh_0_3_point = ds_raw.get('HLCY_P0_2L103_GLC0')[1][grid_points[i,0], grid_points[i,1]]
                srh_0_3_sum = srh_0_3_sum + srh_0_3_point
                
                # Increase number of points with minimal reflectivity by 1
                number_points += 1
                
        # Calculate average over number of grid points and round output
        pwat_avg = float(np.round((pwat_sum / number_points), 4)) # kg/m^2
        mslp_avg = float(np.round(((mslp_sum / number_points) / 100), 4)) # hPa
        cape_avg = float(np.round((cape_sum / number_points), 4)) # J/kg
        cin_avg = float(np.round((cin_sum / number_points), 4)) # J/kg
        t2m_avg = float(np.round(((t2m_sum / number_points) - 273.15), 4)) # deg C
        rh2m_avg = float(np.round((rh2m_sum / number_points), 4)) # percent
        q2m_avg = float(np.round(((q2m_sum / number_points) * 1000), 4)) # g/kg
        u10_avg = float(np.round((u10_sum / number_points), 4)) # m/s
        v10_avg = float(np.round((v10_sum / number_points), 4)) # m/s
        uv10_avg = float(np.round((uv10_sum / number_points), 4)) # m/s
        lcl_avg = np.round(((lcl_sum.magnitude) / number_points), 4) # hPa
        shr0_6_avg = float(np.round((shear_0_6_sum / number_points), 4)) # 1/s
        srh0_1_avg = float(np.round((srh_0_1_sum / number_points), 4)) # m^2/s^2
        srh0_3_avg = float(np.round((srh_0_3_sum / number_points), 4)) # m^2/s^2
        
        # Set time variables equal to ints
        year = int(year)
        month = int(month)
        day = int(day)
        hour = int(hour)
            
        # Export outputs into a csv filter
        data = [year, month, day, hour, label, pwat_avg, mslp_avg, cape_avg, cin_avg, t2m_avg, rh2m_avg, q2m_avg, u10_avg, v10_avg,
                uv10_avg, lcl_avg, shr0_6_avg, srh0_1_avg, srh0_3_avg]
        
        with open("central_ok_mds_env_FINAL.csv", 'a', newline='') as output_file:
            output_writer = writer(output_file)
            output_writer.writerow(data)
            output_file.close()
        
        input_csv_row += 1
    
    # Input MD data so manual inspection of RAP analysis can be done
    # Files can be found at https://www.ncei.noaa.gov/products/weather-climate-models/rapid-refresh-update
    else:
        year = int(year)
        month = int(month)
        day = int(day)
        hour = int(hour)
        
        data = [year, month, day, hour, label, placeholder, placeholder, placeholder, placeholder, placeholder, placeholder, placeholder, 
                placeholder, placeholder, placeholder, placeholder, placeholder, placeholder, placeholder]
        
        # Export outputs into a csv filter
        with open("central_ok_mds_env_FINAL.csv", 'a', newline='') as output_file:
            output_writer = writer(output_file)
            output_writer.writerow(data)
            output_file.close()
            
        input_csv_row += 1